# Loading Data from NHC Hurricane Archive 

Data Availability: [https://www.nhc.noaa.gov/data/hurdat/]
- See format details in `format.pdf`

There are two types of rows for this dataset
- Type 1 Format: entries, date, time,
- Type 2 Format: record_identifier, status, lat, lon, vmax, pres, 34ne, 34se, 34sw, 34nw, 50ne, 50se, 50sw, 50nw, 64ne, 64se, 64sw, 64nw, rmax
   - Note: Date is YYYMMDD and Time is UTC

A note on missing values
- Radii at 34k, 50k, and 64k were not recorded until hurricane AL012004 - ALEX in 07/31/2004 (can be used with HRRR, since HRRR starts in 2014)
- Radii at maximum windspeed (Rmax) was not recorded in many hurricanes until AL012021 - ANA in 05/20/2021 (may require removal or estimation)

In [266]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import subprocess
import pygrib
from cartopy import crs as ccrs, feature as cfeature
import csv

In [267]:
filename = 'hurdat2-1851-2023-051124.txt'
missingVal = -999
basin='AL'
minDate = 20140730
columnHeaders = ['id', 'name', 'entries', 'date', 'time', 'record_identifier', 'status', 'lat', 'lon', 'vmax', 'pres', '34ne', '34se', '34sw', '34nw', '50ne', '50se', '50sw', '50nw', '64ne', '64se', '64sw', '64nw', 'rmax']

First, we will extract minumum/maximum latitude and longitude values from HRRR

- Note: See `loadHRRR.ipynb` for a guide on how HRRR data can be loaded using the AWS CLI

In [304]:
# Open the GRIB2 file
grbs = pygrib.open('hrrr.20210101/conus/hrrr.t18z.wrfnatf00.grib2')

# Select the relevant message (e.g., the first message)
grb = grbs.message(1)

# Extract minimum and maximum latitude and longitude values
values, lats, lons = grb.data()
values.shape, lats.min(), lats.max(), lons.min(), lons.max()


((1059, 1799),
 21.138,
 52.61565399063001,
 -134.09612674048225,
 -60.91783900635959)

Below are several options for datasets constructed using NHC Hurricane Archive data

In [305]:
# Reformatting input file to csv and creating a new file named 'data.csv'
with open(filename, 'r') as f_in, open('data.csv', 'w') as f_out:
    tmp = ""
    for line in f_in:
        line = line.replace(' ', '')  # Remove spaces
        if line.startswith(basin):
            tmp = line.strip()  # Store the basin line
        else:
            if int(line[:8]) >= minDate: # Filter by minimum HRRR data
                line = tmp + line  # Prepend the last stored basin
                f_out.write(line)  # Write filtered line 

In [325]:
data = pd.read_csv('data.csv', header=None, names=columnHeaders) 
data

,id,name,entries,date,time,record_identifier,status,lat,lon,vmax,...,34nw,50ne,50se,50sw,50nw,64ne,64se,64sw,64nw,rmax
0,AL032014,BERTHA,47,20140730,0,NaN,LO,9.6N,41.5W,30,...,0,0,0,0,0,0,0,0,0,-999
1,AL032014,BERTHA,47,20140730,600,NaN,LO,9.7N,43.0W,30,...,0,0,0,0,0,0,0,0,0,-999
2,AL032014,BERTHA,47,20140730,1200,NaN,LO,9.8N,44.7W,30,...,0,0,0,0,0,0,0,0,0,-999
3,AL032014,BERTHA,47,20140730,1800,NaN,LO,10.0N,46.4W,30,...,0,0,0,0,0,0,0,0,0,-999
4,AL032014,BERTHA,47,20140731,0,NaN,LO,10.4N,48.0W,35,...,40,0,0,0,0,0,0,0,0,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5593,AL212023,TWENTY-ONE,6,20231023,1800,NaN,TD,11.5N,83.2W,25,...,0,0,0,0,0,0,0,0,0,60
5594,AL212023,TWENTY-ONE,6,20231024,0,NaN,TD,12.2N,83.4W,25,...,0,0,0,0,0,0,0,0,0,60
5595,AL212023,TWENTY-ONE,6,20231024,130,L,TD,12.4N,83.5W,25,...,0,0,0,0,0,0,0,0,0,60
5596,AL212023,TWENTY-ONE,6,20231024,600,NaN,TD,13.0N,83.8W,25,...,0,0,0,0,0,0,0,0,0,60


Filtering based on HRRR latitude and longitude boundaries

In [326]:
# Convert latitude

data['lat'] = data['lat'].str[:-1].astype(float)


# Convert longitude
data['lon'] = '-' + data['lon'] 
data['lon'] = data['lon'].str[:-1].astype(float)

In [328]:
data = data[data['lat']>lats.min()]
data = data[data['lat']<lats.max()]
data = data[data['lon']<lons.max()]
data = data[data['lon']<lons.max()]
data.shape

(1916, 24)

In [329]:
# Count the number of hurricane tracks
print(f'Number of hurricane tracks available: {data['id'].nunique()}')
data

Number of hurricane tracks available: 106


,id,name,entries,date,time,record_identifier,status,lat,lon,vmax,...,34nw,50ne,50se,50sw,50nw,64ne,64se,64sw,64nw,rmax
18,AL032014,BERTHA,47,20140803,1200,NaN,TS,21.4,-71.6,40,...,50,0,0,0,0,0,0,0,0,-999
19,AL032014,BERTHA,47,20140803,1400,L,TS,21.8,-71.9,40,...,50,0,0,0,0,0,0,0,0,-999
20,AL032014,BERTHA,47,20140803,1800,NaN,TS,22.7,-72.5,45,...,50,0,0,0,0,0,0,0,0,-999
21,AL032014,BERTHA,47,20140804,0,NaN,TS,24.1,-73.1,55,...,40,40,40,0,0,0,0,0,0,-999
22,AL032014,BERTHA,47,20140804,600,NaN,TS,25.4,-73.5,60,...,40,40,40,0,0,0,0,0,0,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5564,AL202023,TAMMY,54,20231025,0,NaN,HU,24.5,-61.2,75,...,80,40,40,30,30,20,20,0,0,20
5574,AL202023,TAMMY,54,20231027,1200,NaN,TS,32.0,-61.0,55,...,180,70,20,20,80,0,0,0,0,40
5575,AL202023,TAMMY,54,20231027,1800,NaN,TS,32.3,-61.4,55,...,180,70,30,30,80,0,0,0,0,40
5576,AL202023,TAMMY,54,20231028,0,NaN,TS,32.6,-61.5,50,...,120,40,0,0,40,0,0,0,0,40


In [330]:
data.to_csv('filtered_data.csv', index=False)

Missing Value Ratios

In [331]:
empty_vmax = (data['vmax'] == -999).sum() / data.shape[0]
empty_pres = (data['pres'] == -999).sum() / data.shape[0]

empty_rmax = (data['rmax'] == -999).sum() / data.shape[0]
empty_34 = (data['34nw'] == -999).sum() / data.shape[0]
empty_50 = (data['50nw'] == -999).sum() / data.shape[0]
empty_64 = (data['64nw'] == -999).sum() / data.shape[0]


print(f"Ratio of missing values:\nvmax: {empty_vmax}\npres: {empty_pres}\nrmax{empty_rmax}\n34: {empty_34}\n50: {empty_50}\n64: {empty_64}")

Ratio of missing values:
vmax: 0.0
pres: 0.0
rmax0.714509394572025
34: 0.0
50: 0.0
64: 0.0


Now, we will an alternate, dictionary dataset approach

In [341]:
# Creating a dictionary - one key for each hurricane - alternate hurDictset
hurDict = {}
newBasin = False

with open('filtered_data.csv', 'r') as f_in:
    reader = csv.reader(f_in)
    next(reader)
    for line in reader:
        if line[0] not in hurDict.keys():
            while len(line[4]) < 4: line[4] = '0' + line[4]
            currDate = pd.to_datetime(line[3] + line[4],format='%Y%m%d%H%M')
            hurDict[line[0]] = [line[1], [(currDate-currDate)/pd.Timedelta('1 hour')], [[line[7][:-1], line[8][:-1]]], [line[9]], [line[10]], [line[11:15]], [line[15:19]], [line[19:23]], [line[23]]]
            
        else:
            while len(line[4]) < 4: line[4] = '0' + line[4]
            date = pd.to_datetime(line[3] + line[4],format='%Y%m%d%H%M')

            if newBasin:
                currDate = date
                newBasin = False

            hurDict[line[0]][1].append((date - currDate)/pd.Timedelta('1 hour')) # Date to first numpy array
            hurDict[line[0]][2].append([line[7][:-1], line[8][:-1]]) # Lat and Lon to fifth numpy array
            hurDict[line[0]][3].append(line[9]) # vmax to sixth numpy array
            hurDict[line[0]][4].append(line[10]) # pres to seventh numpy array
            hurDict[line[0]][5].append(line[11:15]) # 34ne, 34se, 34sw, 34nw to eigth numpy array
            hurDict[line[0]][6].append(line[15:19]) # 50ne, 50se, 50sw, 50nw to ninth numpy array
            hurDict[line[0]][7].append(line[19:23]) # 64ne, 64se, 64sw, 64nw to tenth numpy array
            hurDict[line[0]][8].append(line[23]) # rmax to eleventh numpy array

In [342]:
for hur in hurDict.keys():
    hurDict[hur][1] = np.array(hurDict[hur][1]).astype('float64')
    hurDict[hur][2] = np.array(hurDict[hur][2]).astype('float64')
    hurDict[hur][3] = np.array(hurDict[hur][3]).astype('float64')
    hurDict[hur][4] = np.array(hurDict[hur][4]).astype('float64')
    hurDict[hur][5] = np.array(hurDict[hur][5]).astype('float64')
    hurDict[hur][6] = np.array(hurDict[hur][6]).astype('float64')
    hurDict[hur][7] = np.array(hurDict[hur][7]).astype('float64')
    hurDict[hur][8] = np.array(hurDict[hur][8]).astype('float64')

In [343]:
df = pd.DataFrame(hurDict)
cols = ['name', 'time', 'trajectory', 'vmax', 'pres', '34', '50', '64', 'rmax']
headers = {}
for i in range(len(cols)):
    headers[i] = cols[i]
df = df.T.rename(columns=headers)

print(f'Number of hurricane tracks available: {df.index.nunique()}')
df

Number of hurricane tracks available: 106


,name,time,trajectory,vmax,pres,34,50,64,rmax
AL032014,BERTHA,"[0.0, 2.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, ...","[[21.0, -71.0], [21.0, -71.0], [22.0, -72.0], ...","[40.0, 40.0, 45.0, 55.0, 60.0, 70.0, 70.0, 65....","[1013.0, 1013.0, 1012.0, 1007.0, 1004.0, 998.0...","[[140.0, 100.0, 0.0, 50.0], [140.0, 100.0, 0.0...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[-999.0, -999.0, -999.0, -999.0, -999.0, -999...."
AL042014,CRISTOBAL,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0,...","[[21.0, -72.0], [22.0, -72.0], [22.0, -72.0], ...","[30.0, 30.0, 35.0, 40.0, 45.0, 45.0, 45.0, 50....","[1005.0, 1004.0, 1002.0, 1001.0, 1001.0, 998.0...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[-999.0, -999.0, -999.0, -999.0, -999.0, -999...."
AL052014,DOLLY,"[0.0, 6.0, 12.0, 18.0, 24.0, 25.0, 28.0, 30.0,...","[[21.0, -93.0], [22.0, -94.0], [22.0, -95.0], ...","[35.0, 40.0, 45.0, 40.0, 40.0, 40.0, 40.0, 35....","[1007.0, 1005.0, 1005.0, 1003.0, 1001.0, 1000....","[[50.0, 70.0, 0.0, 0.0], [70.0, 100.0, 0.0, 0....","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[-999.0, -999.0, -999.0, -999.0, -999.0, -999...."
AL072014,FAY,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0,...","[[22.0, -61.0], [22.0, -62.0], [23.0, -63.0], ...","[35.0, 35.0, 35.0, 40.0, 50.0, 55.0, 60.0, 60....","[1007.0, 1006.0, 1005.0, 1003.0, 999.0, 994.0,...","[[120.0, 0.0, 0.0, 120.0], [120.0, 0.0, 0.0, 1...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[-999.0, -999.0, -999.0, -999.0, -999.0, -999...."
AL082014,GONZALO,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0,...","[[21.0, -66.0], [22.0, -67.0], [23.0, -67.0], ...","[115.0, 115.0, 115.0, 110.0, 110.0, 115.0, 125...","[956.0, 953.0, 949.0, 953.0, 953.0, 948.0, 940...","[[100.0, 60.0, 70.0, 100.0], [100.0, 60.0, 70....","[[30.0, 30.0, 30.0, 40.0], [30.0, 30.0, 30.0, ...","[[20.0, 15.0, 15.0, 20.0], [20.0, 20.0, 15.0, ...","[-999.0, -999.0, -999.0, -999.0, -999.0, -999...."
...,...,...,...,...,...,...,...,...,...
AL102023,IDALIA,"[0.0, 48.0, 54.0, 60.0, 66.0, 72.0, 78.0, 84.0...","[[21.0, -86.0], [21.0, -85.0], [21.0, -85.0], ...","[25.0, 60.0, 60.0, 65.0, 70.0, 80.0, 90.0, 105...","[1006.0, 986.0, 984.0, 981.0, 978.0, 973.0, 96...","[[0.0, 0.0, 0.0, 0.0], [80.0, 120.0, 50.0, 50....","[[0.0, 0.0, 0.0, 0.0], [30.0, 40.0, 20.0, 20.0...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[70.0, 20.0, 20.0, 15.0, 15.0, 15.0, 10.0, 10...."
AL132023,LEE,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0,...","[[21.0, -61.0], [22.0, -61.0], [22.0, -62.0], ...","[105.0, 105.0, 105.0, 105.0, 100.0, 100.0, 100...","[954.0, 950.0, 950.0, 948.0, 948.0, 948.0, 948...","[[150.0, 140.0, 100.0, 140.0], [150.0, 140.0, ...","[[90.0, 70.0, 50.0, 80.0], [110.0, 80.0, 60.0,...","[[50.0, 35.0, 30.0, 45.0], [65.0, 40.0, 40.0, ...","[15.0, 15.0, 15.0, 15.0, 15.0, 25.0, 25.0, 30...."
AL162023,OPHELIA,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0,...","[[28.0, -76.0], [29.0, -75.0], [29.0, -75.0], ...","[30.0, 30.0, 35.0, 40.0, 50.0, 60.0, 60.0, 60....","[1012.0, 1012.0, 1008.0, 1002.0, 996.0, 989.0,...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[150.0, 150.0, 120.0, 100.0, 90.0, 60.0, 40.0,..."
AL172023,PHILIPPE,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0]","[[21.0, -65.0], [22.0, -65.0], [23.0, -66.0], ...","[40.0, 35.0, 35.0, 40.0, 45.0, 45.0, 45.0, 45.0]","[1004.0, 1004.0, 1004.0, 1004.0, 1004.0, 1004....","[[110.0, 150.0, 0.0, 0.0], [100.0, 180.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [...","[100.0, 100.0, 100.0, 90.0, 80.0, 80.0, 90.0, ..."


One more alternative, grouping all radii together

In [346]:
# Creating a dictionary - one key for each hurricane - alternate hurDictset
hurDict = {}
newBasin = False

with open('filtered_data.csv', 'r') as f_in:
    reader = csv.reader(f_in)
    next(reader)
    for line in reader:
        if line[0] not in hurDict.keys():
            while len(line[4]) < 4: line[4] = '0' + line[4]
            currDate = pd.to_datetime(line[3] + line[4],format='%Y%m%d%H%M')
            hurDict[line[0]] = [line[1], [(currDate-currDate)/pd.Timedelta('1 hour')], [[line[7][:-1], line[8][:-1]]], [line[9]], [line[10]], [line[11:23]], [line[23]]]
            
        else:
            while len(line[4]) < 4: line[4] = '0' + line[4]
            date = pd.to_datetime(line[3] + line[4],format='%Y%m%d%H%M')

            if newBasin:
                currDate = date
                newBasin = False

            hurDict[line[0]][1].append((date - currDate)/pd.Timedelta('1 hour')) # Date to first numpy array
            hurDict[line[0]][2].append([line[7][:-1], line[8][:-1]]) # Lat and Lon to second numpy array
            hurDict[line[0]][3].append(line[9]) # vmax to third numpy array
            hurDict[line[0]][4].append(line[10]) # pres to fourth numpy array
            hurDict[line[0]][5].append(line[11:23]) # All ne,se,sw,nw to fifth numpy array
            hurDict[line[0]][6].append(line[23]) # rmax to sixth numpy array

In [347]:
for hur in hurDict.keys():
    hurDict[hur][1] = np.array(hurDict[hur][1]).astype('float64')
    hurDict[hur][2] = np.array(hurDict[hur][2]).astype('float64')
    hurDict[hur][3] = np.array(hurDict[hur][3]).astype('float64')
    hurDict[hur][4] = np.array(hurDict[hur][4]).astype('float64')
    hurDict[hur][5] = np.array(hurDict[hur][5]).astype('float64')
    hurDict[hur][6] = np.array(hurDict[hur][6]).astype('float64')

In [348]:
df = pd.DataFrame(hurDict)
cols = ['name', 'time', 'trajectory', 'vmax', 'pres', 'radii', 'rmax']
headers = {}
for i in range(len(cols)):
    headers[i] = cols[i]
df = df.T.rename(columns=headers)

print(f'Number of hurricane tracks available: {df.index.nunique()}')
df

Number of hurricane tracks available: 106


,name,time,trajectory,vmax,pres,radii,rmax
AL032014,BERTHA,"[0.0, 2.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, ...","[[21.0, -71.0], [21.0, -71.0], [22.0, -72.0], ...","[40.0, 40.0, 45.0, 55.0, 60.0, 70.0, 70.0, 65....","[1013.0, 1013.0, 1012.0, 1007.0, 1004.0, 998.0...","[[140.0, 100.0, 0.0, 50.0, 0.0, 0.0, 0.0, 0.0,...","[-999.0, -999.0, -999.0, -999.0, -999.0, -999...."
AL042014,CRISTOBAL,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0,...","[[21.0, -72.0], [22.0, -72.0], [22.0, -72.0], ...","[30.0, 30.0, 35.0, 40.0, 45.0, 45.0, 45.0, 50....","[1005.0, 1004.0, 1002.0, 1001.0, 1001.0, 998.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-999.0, -999.0, -999.0, -999.0, -999.0, -999...."
AL052014,DOLLY,"[0.0, 6.0, 12.0, 18.0, 24.0, 25.0, 28.0, 30.0,...","[[21.0, -93.0], [22.0, -94.0], [22.0, -95.0], ...","[35.0, 40.0, 45.0, 40.0, 40.0, 40.0, 40.0, 35....","[1007.0, 1005.0, 1005.0, 1003.0, 1001.0, 1000....","[[50.0, 70.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[-999.0, -999.0, -999.0, -999.0, -999.0, -999...."
AL072014,FAY,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0,...","[[22.0, -61.0], [22.0, -62.0], [23.0, -63.0], ...","[35.0, 35.0, 35.0, 40.0, 50.0, 55.0, 60.0, 60....","[1007.0, 1006.0, 1005.0, 1003.0, 999.0, 994.0,...","[[120.0, 0.0, 0.0, 120.0, 0.0, 0.0, 0.0, 0.0, ...","[-999.0, -999.0, -999.0, -999.0, -999.0, -999...."
AL082014,GONZALO,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0,...","[[21.0, -66.0], [22.0, -67.0], [23.0, -67.0], ...","[115.0, 115.0, 115.0, 110.0, 110.0, 115.0, 125...","[956.0, 953.0, 949.0, 953.0, 953.0, 948.0, 940...","[[100.0, 60.0, 70.0, 100.0, 30.0, 30.0, 30.0, ...","[-999.0, -999.0, -999.0, -999.0, -999.0, -999...."
...,...,...,...,...,...,...,...
AL102023,IDALIA,"[0.0, 48.0, 54.0, 60.0, 66.0, 72.0, 78.0, 84.0...","[[21.0, -86.0], [21.0, -85.0], [21.0, -85.0], ...","[25.0, 60.0, 60.0, 65.0, 70.0, 80.0, 90.0, 105...","[1006.0, 986.0, 984.0, 981.0, 978.0, 973.0, 96...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[70.0, 20.0, 20.0, 15.0, 15.0, 15.0, 10.0, 10...."
AL132023,LEE,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0,...","[[21.0, -61.0], [22.0, -61.0], [22.0, -62.0], ...","[105.0, 105.0, 105.0, 105.0, 100.0, 100.0, 100...","[954.0, 950.0, 950.0, 948.0, 948.0, 948.0, 948...","[[150.0, 140.0, 100.0, 140.0, 90.0, 70.0, 50.0...","[15.0, 15.0, 15.0, 15.0, 15.0, 25.0, 25.0, 30...."
AL162023,OPHELIA,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0,...","[[28.0, -76.0], [29.0, -75.0], [29.0, -75.0], ...","[30.0, 30.0, 35.0, 40.0, 50.0, 60.0, 60.0, 60....","[1012.0, 1012.0, 1008.0, 1002.0, 996.0, 989.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[150.0, 150.0, 120.0, 100.0, 90.0, 60.0, 40.0,..."
AL172023,PHILIPPE,"[0.0, 6.0, 12.0, 18.0, 24.0, 30.0, 36.0, 42.0]","[[21.0, -65.0], [22.0, -65.0], [23.0, -66.0], ...","[40.0, 35.0, 35.0, 40.0, 45.0, 45.0, 45.0, 45.0]","[1004.0, 1004.0, 1004.0, 1004.0, 1004.0, 1004....","[[110.0, 150.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[100.0, 100.0, 100.0, 90.0, 80.0, 80.0, 90.0, ..."
